<a href="https://colab.research.google.com/github/babcockt18/qosf-screener/blob/main/QOSF_Screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Task 1: Less Than K

##Problem Statement

Given a positive integer “k” and a list of integer numbers, look for the numbers within the list, that are less than k.



## Install Libraries and Validate Compute Environment

The below cell confirms nvidia compatability of GPU, then installs and configures cuda statevec v12, cuQuantum SDK, and PennyLane's integration with cuQuantum

In [ ]:
!nvidia-smi
!pip -qqq install wheel custatevec-cu12
!export CUQUANTUM_SDK=$(python -c "import site; print( f'{site.getsitepackages()[0]}/cuquantum/lib')")
!pip install -qqq pennylane-lightning[gpu]

Tue Mar 26 02:35:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Import Libraries and Validate Proper Import and Installation.

Install PennyLane and PennyLane's version of numpy, then validate that the import has correctly imported the PennyLane Lightning GPU integraton.

In [ ]:
import pennylane as qml
from pennylane import numpy as np

In [ ]:
qml.about()

Name: PennyLane
Version: 0.35.1
Summary: PennyLane is a cross-platform Python library for quantum computing, quantum machine learning, and quantum chemistry. Train a quantum computer the same way as a neural network.
Home-page: https://github.com/PennyLaneAI/pennylane
Author: 
Author-email: 
License: Apache License 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: appdirs, autograd, autoray, cachetools, networkx, numpy, pennylane-lightning, requests, rustworkx, scipy, semantic-version, toml, typing-extensions
Required-by: PennyLane_Lightning, PennyLane_Lightning_GPU

Platform info:           Linux-6.1.58+-x86_64-with-glibc2.35
Python version:          3.10.12
Numpy version:           1.25.2
Scipy version:           1.11.4
Installed devices:
- lightning.gpu (PennyLane_Lightning_GPU-0.35.1)
- lightning.qubit (PennyLane_Lightning-0.35.1)
- default.clifford (PennyLane-0.35.1)
- default.gaussian (PennyLane-0.35.1)
- default.mixed (PennyLane-0.35.1)
- default.qubit (PennyLane-0


## Define the Custom Oracle

We decide to leverage Grover's Algorithm to solve task 1. Based on the problem statement, we define the oracle function that encodes the problem of finding numbers less than k in list_n. It converts each number to its binary representation and applies X gates to qubits corresponding to 1's in the binary representation. Then, it applies a multi-controlled X gate to mark the states that satisfy the condition (binary_num < k).

In [ ]:
def oracle(k, list_n):
    """
    Oracle function that encodes the problem of finding numbers less than k in list_n.

    Args:
        k (int): The target number to compare against.
        list_n (list): The list of numbers to search for numbers less than k.

    Returns:
        function: The oracle function that marks the states satisfying the condition.
    """
    def _oracle(wires):
        # Convert the numbers to binary representation
        binary_numbers = [np.binary_repr(num, width=len(wires)) for num in list_n]

        # Iterate over each number in the list
        for binary_num in binary_numbers:
            # Apply X gates to qubits corresponding to 1's in the binary representation
            for i, bit in enumerate(binary_num):
                if bit == '1':
                    qml.PauliX(wires=wires[i])

            # Apply a multi-controlled X gate to mark the states that satisfy the condition
            if int(binary_num, 2) < k:
                qml.MultiControlledX(wires=wires[:-1])

            # Apply X gates again to revert the qubits to their original state
            for i, bit in enumerate(binary_num):
                if bit == '1':
                    qml.PauliX(wires=wires[i])

    return _oracle

## Implement Grover's Search Algorithm

We define the less_than_k function that implements Grover's search algorithm. It applies Hadamard gates to all qubits, applies the oracle, and then we apply PennyLane's diffusion operator, [qml.templates.GroverOperator](https://docs.pennylane.ai/en/stable/code/api/pennylane.GroverOperator.html).

*Note: the function does not currently handle duplicate values within the list of numbers to search through. This functionality is forthcoming.*

In [ ]:
def less_than_k(k, list_n):
    """
    Find numbers less than k in the given list using Grover's search algorithm.

    Args:
        k (int): The target number to compare against.
        list_n (list): The list of numbers to search for numbers less than k.

    Returns:
        tuple: A tuple containing two elements:
            - list: The sorted list of numbers less than k found in list_n.
            - function: The quantum circuit representation of Grover's search algorithm.
    """
    # Determine the number of qubits required
    num_qubits = len(bin(max(list_n))[2:]) + 1

    # Create a quantum device
    dev = qml.device('lightning.gpu', wires=num_qubits, shots=1000)

    # Define the quantum circuit for Grover's search
    @qml.qnode(dev, interface='auto')
    def grover_circuit(k, list_n):
        # Apply Hadamard gates to all qubits
        for i in range(num_qubits):
            qml.Hadamard(wires=i)

        # Apply the oracle
        oracle_func = oracle(k, list_n)
        oracle_func(wires=range(num_qubits))

        # Apply the diffusion operator
        qml.templates.GroverOperator(wires=range(num_qubits - 1))

        return qml.sample(wires=range(num_qubits - 1))

    # Run the Grover's search circuit
    samples = grover_circuit(k, list_n)

    # Interpret the measurement results
    results = []
    for sample in samples:
        binary_string = ''.join(str(bit) for bit in sample)
        number = int(binary_string, 2)
        if number < len(list_n) and list_n[number] < k:
            results.append(list_n[number])

    return sorted(set(results)), qml.draw(grover_circuit)(k, list_n)

In [ ]:
# Example usage
A = less_than_k(5, [-1, 4, 9, 11, 14, 1, 13, 6, 3])

print('Circuit Architecture:\n\n' + A[1] + f'\n\n Answer:\n\n {A[0]}')

Circuit Architecture:

0: ──H──X──X────╭●───────────────────╭●─────────────╭●────╭GroverOperator─┤ ╭Sample
1: ──H──X──X────├●──X──X──X──X──X──X─├●──X──X───────├●────├GroverOperator─┤ ├Sample
2: ──H──X──X──X─├●──X──X──X──────────├●──X──X──X──X─├●────├GroverOperator─┤ ├Sample
3: ──H──X──X────╰X──X──X──X──X───────╰X──X──X──X────╰X──X─╰GroverOperator─┤ ╰Sample
4: ──H──X──X──X──X──X──X──X──X──X──X──X──X────────────────────────────────┤        

 Answer:

 [-1, 1, 3, 4]


In [ ]:
# TODO: figure out how to handle duplicate values properly
# TODO: Consider encoding scheme to handle a certain decimal place
# TODO: Consider How to Optimize gates. There are a lot of X gates that ostensibly look like low-hanging fruit